In [1]:
'''
Note, la fonction de nettoyage va convertir toutes les colonnes en str, MtSup ne fera plus la diff entre des listes et des dict 
 '''

'\nNote, la fonction de nettoyage va convertir toutes les colonnes en str, MtSup ne fera plus la diff entre des listes et des dict \n '

Objo : 
- lancer le test avec la nouvelle task1 sur les 20k 
- retravailler mtsup et caracsup 

In [1]:
import polars as pl 
import gzip 
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
#Fonctions de base

def _recherche_id_dans_parquet(chemin_parquet_db):
 
 conduck = duckdb.connect(database=':memory:', read_only=False)
 source_duck = conduck.read_parquet(f'{chemin_parquet_db}')
 requete = conduck.execute(''' 
    SELECT DISTINCT Id_Fichier 
    FROM source_duck''')
 ligne_sql_int = [int(x[0]) for x in requete.fetchall()]
 conduck.close()
 return ligne_sql_int

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 ''' Pas encore utilisée, permet de récupérer les ID dans le parquet contenant les 
 données déjà traitées '''
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

def nettoyage_V(dataframe : pd.DataFrame) -> pd.DataFrame :
 ''' Permet de supprimer les @V des colonnes à l'exception de MtSup et CaracSup'''

 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['MtSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre

def transfo_mtsup_if_dict(ligne_budget_dict : dict) -> dict:
 ''' Traite les MtSup, qu'ils soient sous forme de list ou de dict '''
 type_mtsup = ligne_budget_dict.get('MtSup')  # Permet de connaitre le type de MtSup
 if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
    if isinstance(type_mtsup, dict):
      dict_mtsup = type_mtsup
      ligne_budget_dict['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
      ligne_budget_dict['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
    elif isinstance(type_mtsup, list):
      dict_mtsup = ligne_budget_dict['MtSup']
      mtsup_propre = {}
      for z, entry in enumerate(dict_mtsup, start=1):
        code = f'MtSup_{z}_Lib'
        valeur = f'MtSup_{z}_Val'
        mtsup_propre[code] = entry.get('@Code', '')
        mtsup_propre[valeur] = entry.get('@V', '')
        ligne_budget_dict.update(mtsup_propre)
 return ligne_budget_dict

def transfo_caracsup_if_budget_dict(ligne_budget_dict : dict) -> dict: 
 type_caracsup = ligne_budget_dict.get('CaracSup')  # Permet de connaitre le type de CaracSup
 if type_caracsup is not None:  # Vérifie si la clé 'CaracSup' existe
    if isinstance(type_caracsup, dict):
      dict_caracsup = type_caracsup
      ligne_budget_dict['CaracSup_1_Lib'] = {'@V': dict_caracsup.get('@Code', '')}
      ligne_budget_dict['CaracSup_1_Val'] = {'@V': dict_caracsup.get('@V', '')}
    elif isinstance(type_caracsup, list):
      dict_caracsup = ligne_budget_dict['CaracSup']
      caracsup_propre = {}
      for z, entry in enumerate(dict_caracsup, start=1):
        code = f'CaracSup_{z}_Lib'
        valeur = f'CaracSup_{z}_Val'
        caracsup_propre[code] = entry.get('@Code', '')
        caracsup_propre[valeur] = entry.get('@V', '')
        ligne_budget_dict.update(caracsup_propre)      
 return ligne_budget_dict

In [4]:
#parquet_schema

schema_parquet = pa.schema([
    ('Id_Fichier', pa.int64()),
    ('IdColl', pa.string()),
    ('DteStr', pa.date32()),
    ('LibellePoste', pa.string()),
    ('IdPost', pa.string()),
    ('TypOpBudg', pa.string()), #des 2 et des 1
    ('OpeCpteTiers', pa.string()),
    ('NatCEPL', pa.string()),
    ('Departement', pa.string()), #On oublie pas les 2A,
    ('NatDec', pa.string()), #A terme, sera un pa.dicitonnary
    ('Exer', pa.int16()),
    ('ProjetBudget', pa.string()), #bool mais à verif 
    ('NatVote', pa.string()), #dico aussi
    ('OpeEquip', pa.string()), #necessite recherche
    ('VoteFormelChap', pa.string()), #bool mais à verif 
    ('TypProv', pa.string()), #int ? 
    ('BudgPrec', pa.string()), #int ?
    ('ReprRes', pa.string()), #int ?
    ('NatFonc', pa.string()), #int ?
    ('CodTypBud', pa.string()),
    ('LibelleEtabPal', pa.string()),
    ('IdEtabPal', pa.string()),
    ('LibelleEtab', pa.string()),
    ('IdEtab', pa.string()),
    ('CodColl', pa.string()),
    ('CodInseeColl', pa.string()),
    ('CodBud', pa.string()),
    ('Nomenclature', pa.string()),
    ('Nature', pa.string()),
    ('Fonction', pa.string()),
    ('LibCpte', pa.string()),
    ('ContNat', pa.string()),
    ('ArtSpe', pa.string()), #Doit être corrigé en bool,
    ('ContFon', pa.string()),
    ('CodRD', pa.string()), #Val D ou R,
    ('MtBudgPrec', pa.float32()), #float32 #nullable = True
    ('MtRARPrec', pa.float32()), #float32
    ('MtPropNouv', pa.float32()), #float32
    ('MtPrev', pa.float32()), #float32
    ('CredOuv', pa.float32()), #int32
    ('MtReal', pa.float32()), #float32
    ('MtRAR3112', pa.float32()), #float32
    ('OpBudg', pa.string()),
    ('MtSup', pa.string()),
    ('CaracSup', pa.string()),
        ('MtSup_1_Lib', pa.string()),
        ('MtSup_1_Val', pa.string()),
        ('MtSup_2_Lib', pa.string()),
        ('MtSup_2_Val', pa.string()),
        ('MtSup_3_Lib', pa.string()),
        ('MtSup_3_Val', pa.string()),
        ('MtSup_4_Lib', pa.string()),
        ('MtSup_4_Val', pa.string()),
        ('MtSup_5_Lib', pa.string()),
        ('MtSup_5_Val', pa.string()),
        ('MtSup_6_Lib', pa.string()),
        ('MtSup_6_Val', pa.string()),
        ('MtSup_7_Lib', pa.string()),
        ('MtSup_7_Val', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        
    ])

In [5]:
def task_1_v2(chemin_xml_entree, chemin_parquet_verif, chemin_sortie) : 
 ''' La V2 traite en plus :
  - F Le cas où il n'y a qu'une seule ligne budget (pour caracsup et mtsup).
  - F Ajoute des chemins que l'on peut choisir
  - F Ajoute une verification depuis un plus grand parquet, ça pourra se faire via une db plus tôt, quand on en aura une
  - F Sert un café robusta pas fou mais buvable
  - F Ajoute de nouvelles métadonnées, dont l'année d'exercice, la nature de vote, le siret d'etab parfois different du premier siret etc. 
  - En cours, Fait le schema à cette étape pour éviter tout soucis après 
 '''
 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_xml_entree, "*.gz"))
 liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree_glob if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in liste_fichiers_safe : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['BlocBudget'])
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['EnTeteBudget'])
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = transfo_mtsup_if_dict(i)
      i = transfo_caracsup_if_budget_dict(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = transfo_mtsup_if_dict(docbase)
    docbase = transfo_caracsup_if_budget_dict(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(f'Erreur complète : {repr(e)}')
    traceback.print_exc()
    continue 

On va prendre un fichier test au pif 

In [3]:
chemin_612013 = '../../fichiers20/todo_xml_20/612013.xml.gz'

In [7]:
dico_612013 = parse_fichier(chemin_612013)

In [8]:
id_612013 = '612013'
metadonnees_612013 = dico_612013['DocumentBudgetaire']['EnTeteDocBudgetaire']
metadonnees_612013.update(dico_612013['DocumentBudgetaire']['Budget']['BlocBudget'])
metadonnees_612013.update(dico_612013['DocumentBudgetaire']['Budget']['EnTeteBudget'])
metadonnees_612013['Id_Fichier'] = {'@V' : id_612013}
docbase_612013 = dico_612013['DocumentBudgetaire']['Budget']['LigneBudget']

for i in docbase_612013 : 
 i.update(metadonnees_612013)

In [9]:
''' Pour Mtsup et carac sup dans l'idée ça donnerait : 
dico_612013
if isinstance(Mtsup : dict ) : 
 for key, value in Mtsup : 
    
elif isinstance(MtSup : list) : '''

" Pour Mtsup et carac sup dans l'idée ça donnerait : \ndico_612013\nif isinstance(Mtsup : dict ) : \n for key, value in Mtsup : \n    \nelif isinstance(MtSup : list) : "

In [10]:
df_612013 = pd.DataFrame(docbase_612013)
df_612013 = nettoyage_V(df_612013)
df_612013

,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,...,CodColl,CodInseeColl,CodBud,Nomenclature,Id_Fichier,TypOpBudg,MtRARPrec,LibCpte,Operation,ContOp
0,60632,011,false,D,11962.00,19335.00,19335.00,19335.00,0,"{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,nan,nan,nan
1,6332,012,false,D,450.00,500.00,500.00,500.00,0,"{'@V': '450.00', '@Code': 'BudgetHorsRAR'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,nan,nan,nan
2,6411,012,false,D,365000.00,352000.00,352000.00,352000.00,0,"{'@V': '365000.00', '@Code': 'BudgetHorsRAR'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,nan,nan,nan
3,74834,nan,false,R,4104.00,4084.00,4084.00,4084.00,0,"{'@V': '4104.00', '@Code': 'BudgetHorsRAR'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,nan,nan,nan
4,6331,012,false,D,4250.00,5200.00,5200.00,5200.00,0,"{'@V': '4250.00', '@Code': 'BudgetHorsRAR'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,1641,nan,false,R,nan,nan,0.00,nan,0,"{'@V': '500000.00', '@Code': 'MtOpeCumul'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,MAISON DE SANTE,2019001,nan
178,2033,nan,false,D,nan,nan,0.00,nan,0,"{'@V': '941.76', '@Code': 'MtOpeCumul'}",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,MAISON DE SANTE,2019001,2019001
179,1323,nan,false,R,25041.00,nan,0.00,nan,0,nan,...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,BIBLIOTHEQUE,2016012,nan
180,2158,nan,false,D,nan,3671.91,3671.91,3671.91,0,"[{'@V': '78134.61', '@Code': 'MtOpeCumul'}, {'...",...,201,24002,00,M14-M14_COM_500_3500,612013,nan,nan,BORNES SEMI ENTERREES,2018001,2018001


In [11]:
df_mtsup = df_612013[['MtSup']]
df_mtsup


,MtSup
0,"{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}"
1,"{'@V': '450.00', '@Code': 'BudgetHorsRAR'}"
2,"{'@V': '365000.00', '@Code': 'BudgetHorsRAR'}"
3,"{'@V': '4104.00', '@Code': 'BudgetHorsRAR'}"
4,"{'@V': '4250.00', '@Code': 'BudgetHorsRAR'}"
...,...
177,"{'@V': '500000.00', '@Code': 'MtOpeCumul'}"
178,"{'@V': '941.76', '@Code': 'MtOpeCumul'}"
179,nan
180,"[{'@V': '78134.61', '@Code': 'MtOpeCumul'}, {'..."


In [4]:
dict612013 = parse_fichier(chemin_612013)
dtest = dict612013['DocumentBudgetaire']['Budget']['LigneBudget']

In [8]:
mtdict = 0
mtlist = 0
mtnan = 0
ldict = []
llist = []
for i in dtest : 
    type_m = i.get('MtSup')  # Permet de connaitre le type de MtSup
    if isinstance(type_m, dict) : 
        mtdict += 1 
        ldict.append(i)
    elif isinstance(type_m, list) : 
        mtlist +=1 
        llist.append(i)
    else : 
        mtnan +=1

print('dict : ', mtdict)
print('list : ', mtlist)
print('nan : ', mtnan)
print('bon si 141,12,29')


dict :  141
list :  12
nan :  29
bon si 141,12,29


In [9]:
#On va conserver que les dict pour le moment 
ldict
aa = 0
bb = 0
zz = 0 

for i in ldict : 
    type_m = i.get('MtSup')  # Permet de connaitre le type de MtSup
    if isinstance(type_m, dict) : 
        aa += 1 
    elif isinstance(type_m, list) : 
        bb +=1 
    else : 
        zz +=1

print('dict : ', aa)
print('list : ', bb)
print('nan : ', zz)
print('bon si 141,0,0')

dict :  141
list :  0
nan :  0
bon si 141,0,0


In [15]:

ldict 

[{'Nature': {'@V': '60632'},
  'ContNat': {'@V': '011'},
  'ArtSpe': {'@V': 'false'},
  'CodRD': {'@V': 'D'},
  'MtBudgPrec': {'@V': '11962.00'},
  'MtPropNouv': {'@V': '19335.00'},
  'MtPrev': {'@V': '19335.00'},
  'CredOuv': {'@V': '19335.00'},
  'OpBudg': {'@V': '0'},
  'MtSup': {'@V': '11962.00', '@Code': 'BudgetHorsRAR'},
  'CaracSup': {'@V': 'F', '@Code': 'Section'}},
 {'Nature': {'@V': '6332'},
  'ContNat': {'@V': '012'},
  'ArtSpe': {'@V': 'false'},
  'CodRD': {'@V': 'D'},
  'MtBudgPrec': {'@V': '450.00'},
  'MtPropNouv': {'@V': '500.00'},
  'MtPrev': {'@V': '500.00'},
  'CredOuv': {'@V': '500.00'},
  'OpBudg': {'@V': '0'},
  'MtSup': {'@V': '450.00', '@Code': 'BudgetHorsRAR'},
  'CaracSup': {'@V': 'F', '@Code': 'Section'}},
 {'Nature': {'@V': '6411'},
  'ContNat': {'@V': '012'},
  'ArtSpe': {'@V': 'false'},
  'CodRD': {'@V': 'D'},
  'MtBudgPrec': {'@V': '365000.00'},
  'MtPropNouv': {'@V': '352000.00'},
  'MtPrev': {'@V': '352000.00'},
  'CredOuv': {'@V': '352000.00'},
  'OpBu

In [16]:
for i in ldict : 
      type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
      dict_mtsup = type_mtsup
      i[dict_mtsup.get('@Code', '')] = {'@V' : dict_mtsup.get('@V', '')}


df_result_ldict = pd.DataFrame(ldict)
df_result_ldict.columns

Index(['Nature', 'ContNat', 'ArtSpe', 'CodRD', 'MtBudgPrec', 'MtPropNouv',
       'MtPrev', 'CredOuv', 'OpBudg', 'MtSup', 'CaracSup', 'BudgetHorsRAR',
       'TypOpBudg', 'MtRARPrec', 'RARPrec', 'LibCpte', 'Operation', 'ContOp',
       'MtOpeInfo', 'MtOpeCumul'],
      dtype='object')

In [17]:
df_result_ldict[['BudgetHorsRAR', 'MtOpeInfo', 'MtOpeCumul']]

,BudgetHorsRAR,MtOpeInfo,MtOpeCumul
0,{'@V': '11962.00'},NaN,NaN
1,{'@V': '450.00'},NaN,NaN
2,{'@V': '365000.00'},NaN,NaN
3,{'@V': '4104.00'},NaN,NaN
4,{'@V': '4250.00'},NaN,NaN
...,...,...,...
136,NaN,NaN,NaN
137,NaN,NaN,{'@V': '2131.92'}
138,NaN,NaN,{'@V': '500000.00'}
139,NaN,NaN,{'@V': '941.76'}


Pour les dict, c'est bon, maintenant les listes. 

In [10]:
len(llist)

elem_decomp = llist[1]
llist[1].get('MtSup') #Contient deux dict

for za in elem_decomp.get('MtSup') : #for i in ze deux dict
 elem_decomp[za.get('@Code', '')] = {'@V' : za.get('@V', '')}

elem_decomp

{'Nature': {'@V': '2188'},
 'ContNat': {'@V': '21'},
 'ArtSpe': {'@V': 'false'},
 'CodRD': {'@V': 'D'},
 'MtBudgPrec': {'@V': '15998.85'},
 'MtRARPrec': {'@V': '3835.57'},
 'MtPropNouv': {'@V': '5664.43'},
 'MtPrev': {'@V': '5664.43'},
 'CredOuv': {'@V': '9500.00'},
 'OpBudg': {'@V': '0'},
 'MtSup': [{'@V': '15000.00', '@Code': 'BudgetHorsRAR'},
  {'@V': '3835.57', '@Code': 'RARPrec'}],
 'CaracSup': {'@V': 'I', '@Code': 'Section'},
 'BudgetHorsRAR': {'@V': '15000.00'},
 'RARPrec': {'@V': '3835.57'}}

In [19]:
test_2key = elem_decomp.copy()

In [21]:
test_2key['test'] = 1
test_2key

{'Nature': {'@V': '2188'},
 'ContNat': {'@V': '21'},
 'ArtSpe': {'@V': 'false'},
 'CodRD': {'@V': 'D'},
 'MtBudgPrec': {'@V': '15998.85'},
 'MtRARPrec': {'@V': '3835.57'},
 'MtPropNouv': {'@V': '5664.43'},
 'MtPrev': {'@V': '5664.43'},
 'CredOuv': {'@V': '9500.00'},
 'OpBudg': {'@V': '0'},
 'MtSup': [{'@V': '15000.00', '@Code': 'BudgetHorsRAR'},
  {'@V': '3835.57', '@Code': 'RARPrec'}],
 'CaracSup': {'@V': 'I', '@Code': 'Section'},
 'BudgetHorsRAR': {'@V': '15000.00'},
 'RARPrec': {'@V': '3835.57'},
 'test': 1}

Probleme : {'@V': '15000.00'} n'est pas considéré comme un dict mais comme un str 

[{'Nature': {'@V': '2111'},
  'ContNat': {'@V': '21'},
  'ArtSpe': {'@V': 'false'},
  'CodRD': {'@V': 'D'},
  'MtBudgPrec': {'@V': '23660.00'},
  'MtRARPrec': {'@V': '19965.00'},
  'MtPropNouv': {'@V': '4000.00'},
  'MtPrev': {'@V': '4000.00'},
  'CredOuv': {'@V': '23965.00'},
  'OpBudg': {'@V': '0'},
  'MtSup': [{'@V': '2040.00', '@Code': 'BudgetHorsRAR'},
   {'@V': '19965.00', '@Code': 'RARPrec'}],
  'CaracSup': {'@V': 'I', '@Code': 'Section'}},
 {'Nature': {'@V': '2188'},
  'ContNat': {'@V': '21'},
  'ArtSpe': {'@V': 'false'},
  'CodRD': {'@V': 'D'},
  'MtBudgPrec': {'@V': '15998.85'},
  'MtRARPrec': {'@V': '3835.57'},
  'MtPropNouv': {'@V': '5664.43'},
  'MtPrev': {'@V': '5664.43'},
  'CredOuv': {'@V': '9500.00'},
  'OpBudg': {'@V': '0'},
  'MtSup': [{'@V': '15000.00', '@Code': 'BudgetHorsRAR'},
   {'@V': '3835.57', '@Code': 'RARPrec'}],
  'CaracSup': {'@V': 'I', '@Code': 'Section'},
  'BudgetHorsRAR': {'@V': '15000.00'},
  'RARPrec': {'@V': '3835.57'}},
 {'Nature': {'@V': '1328'},

In [24]:
def test_mtsup(ligne_bdg) : 

 format_mt = ligne_bdg.get('MtSup')

 if isinstance(format_mt, dict) : 
  mtsup_si_dict = ligne_bdg.get('MtSup')
  dict_mtsup = mtsup_si_dict
  ligne_bdg[dict_mtsup.get('@Code', '')] = {'@V' : dict_mtsup.get('@V', '')}

 elif isinstance(format_mt, list) : 
  for i in format_mt : 
    for za in i.get('MtSup') : #for i in ze deux dict
     i[za.get('@Code', '')] = {'@V' : za.get('@V', '')}

 else : 
  pass
 return ligne_bdg 



def testv2mtsup(bdgs) :
 ndict = 0
 nlist = 0 
 for i in bdgs : 
  
    type_de_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup

    if isinstance(type_de_mtsup, dict) : 
      mtsup_vdict = type_de_mtsup
      i[mtsup_vdict.get('@Code', '')] = {'@V' : mtsup_vdict.get('@V', '')}

    elif isinstance(type_de_mtsup, list) : 
     elem_decomp = bdgs[i]
     bdgs[i].get('MtSup') #Contient deux dict
     for za in bdgs[i].get('MtSup') : #for i in ze deux dict
      bdgs[i][za.get('@Code', '')] = {'@V' : za.get('@V', '')}  
    else : 
        pass
      
 return bdgs

In [25]:
dico_test_total = parse_fichier(chemin_612013)
lbgs_test = dico_test_total['DocumentBudgetaire']['Budget']['LigneBudget']
testv2mtsup(lbgs_test)

TypeError: list indices must be integers or slices, not dict

In [43]:
df_test1 = nettoyage_V(df_test)
df_test1.head(3)


,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,CaracSup,BudgetHorsRAR,TypOpBudg,MtRARPrec,RARPrec,LibCpte,Operation,ContOp,MtOpeInfo,MtOpeCumul
0,60632,011,false,D,11962.00,19335.00,19335.00,19335.00,0,"{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",11962.00,nan,nan,nan,nan,nan,nan,nan,nan
1,6332,012,false,D,450.00,500.00,500.00,500.00,0,"{'@V': '450.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",450.00,nan,nan,nan,nan,nan,nan,nan,nan
2,6411,012,false,D,365000.00,352000.00,352000.00,352000.00,0,"{'@V': '365000.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",365000.00,nan,nan,nan,nan,nan,nan,nan,nan


In [58]:
df_test.head(1)

,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,CaracSup,BudgetHorsRAR,TypOpBudg,MtRARPrec,RARPrec,LibCpte,Operation,ContOp,MtOpeInfo,MtOpeCumul
0,{'@V': '60632'},{'@V': '011'},{'@V': 'false'},{'@V': 'D'},{'@V': '11962.00'},{'@V': '19335.00'},{'@V': '19335.00'},{'@V': '19335.00'},{'@V': '0'},"{'@V': '11962.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",{'@V': '11962.00'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
dt_list = df_test[df_test['MtSup'].apply(lambda x: isinstance(x, list))]

In [64]:
dt_list

,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,CaracSup,BudgetHorsRAR,TypOpBudg,MtRARPrec,RARPrec,LibCpte,Operation,ContOp,MtOpeInfo,MtOpeCumul
15,{'@V': '2111'},{'@V': '21'},{'@V': 'false'},{'@V': 'D'},{'@V': '23660.00'},{'@V': '4000.00'},{'@V': '4000.00'},{'@V': '23965.00'},{'@V': '0'},"[{'@V': '2040.00', '@Code': 'BudgetHorsRAR'}, ...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '19965.00'},NaN,NaN,NaN,NaN,NaN,NaN
33,{'@V': '2188'},{'@V': '21'},{'@V': 'false'},{'@V': 'D'},{'@V': '15998.85'},{'@V': '5664.43'},{'@V': '5664.43'},{'@V': '9500.00'},{'@V': '0'},"[{'@V': '15000.00', '@Code': 'BudgetHorsRAR'},...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '3835.57'},NaN,NaN,NaN,NaN,NaN,NaN
77,{'@V': '1328'},NaN,{'@V': 'false'},{'@V': 'R'},{'@V': '2964.00'},NaN,{'@V': '0.00'},{'@V': '2964.00'},{'@V': '0'},"[{'@V': '2964.00', '@Code': 'BudgetHorsRAR'}, ...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '2964.00'},NaN,NaN,NaN,NaN,NaN,NaN
84,{'@V': '2183'},{'@V': '21'},{'@V': 'false'},{'@V': 'D'},{'@V': '6000.00'},NaN,{'@V': '0.00'},{'@V': '7000.00'},{'@V': '0'},"[{'@V': '6000.00', '@Code': 'BudgetHorsRAR'}, ...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '7000.00'},NaN,NaN,NaN,NaN,NaN,NaN
105,{'@V': '2313'},{'@V': '23'},{'@V': 'false'},{'@V': 'D'},{'@V': '43800.00'},{'@V': '98180.90'},{'@V': '98180.90'},{'@V': '193487.11'},{'@V': '0'},"[{'@V': '23300.00', '@Code': 'BudgetHorsRAR'},...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '95306.21'},NaN,NaN,NaN,NaN,NaN,NaN
125,{'@V': '2158'},{'@V': '21'},{'@V': 'false'},{'@V': 'D'},{'@V': '2300.00'},{'@V': '-2371.91'},{'@V': '-2371.91'},{'@V': '1300.00'},{'@V': '0'},"[{'@V': '2300.00', '@Code': 'BudgetHorsRAR'}, ...","{'@V': 'I', '@Code': 'Section'}",NaN,NaN,{'@V': '3671.91'},NaN,NaN,NaN,NaN,NaN,NaN
156,{'@V': '1341'},NaN,{'@V': 'false'},{'@V': 'R'},{'@V': '222593.54'},NaN,{'@V': '0.00'},{'@V': '151315.31'},{'@V': '0'},"[{'@V': '71278.23', '@Code': 'MtOpeCumul'}, {'...","[{'@V': '1', '@Code': 'TypOpe'}, {'@V': 'I', '...",NaN,NaN,{'@V': '151315.31'},NaN,{'@V': 'NOUVELLE ECOLE MATERNELLE ET SES ABORDS'},{'@V': '2017002'},NaN,NaN,NaN
158,{'@V': '1321'},NaN,{'@V': 'false'},{'@V': 'R'},{'@V': '5000.00'},NaN,{'@V': '0.00'},{'@V': '57343.20'},{'@V': '0'},"[{'@V': '5000.00', '@Code': 'BudgetHorsRAR'}, ...","[{'@V': '1', '@Code': 'TypOpe'}, {'@V': 'I', '...",NaN,NaN,{'@V': '57343.20'},NaN,{'@V': 'NOUVELLE ECOLE MATERNELLE ET SES ABORDS'},{'@V': '2017002'},NaN,NaN,NaN
159,{'@V': '2313'},NaN,{'@V': 'false'},{'@V': 'D'},{'@V': '1152768.40'},{'@V': '173359.00'},{'@V': '173359.00'},{'@V': '410392.78'},{'@V': '0'},"[{'@V': '1197939.05', '@Code': 'MtOpeCumul'}, ...","[{'@V': '1', '@Code': 'TypOpe'}, {'@V': 'I', '...",NaN,NaN,{'@V': '237033.78'},NaN,{'@V': 'NOUVELLE ECOLE MATERNELLE ET SES ABORDS'},{'@V': '2017002'},{'@V': '2017002'},NaN,NaN
169,{'@V': '2315'},NaN,{'@V': 'false'},{'@V': 'D'},{'@V': '87264.00'},{'@V': '59132.38'},{'@V': '59132.38'},{'@V': '59132.38'},{'@V': '0'},"[{'@V': '60316.00', '@Code': 'BudgetHorsRAR'},...","[{'@V': '1', '@Code': 'TypOpe'}, {'@V': 'I', '...",NaN,NaN,NaN,NaN,{'@V': 'VRD GYMNASE'},{'@V': '2017001'},{'@V': '2017001'},NaN,NaN
